In [ ]:
import os
import csv

def flatten_classification_report(report):
    flattened_report = {}
    for key, value in report.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                flattened_report[f"{key}_{sub_key}"] = sub_value
        else:
            flattened_report[key] = value
    return flattened_report

def evaluate_and_save_classification_results(model_name, y_test, y_pred, output_file="classification_results.csv"):
    confusion_matrix_result = confusion_matrix(y_test, y_pred)
    classification_report_result = classification_report(y_test, y_pred, output_dict=True)

    
    # extract features from confusion matrix
    tn, fp, fn, tp = confusion_matrix_result.ravel()
    
    # flatten the classification report
    flattened_report = flatten_classification_report(classification_report_result)
    
    # add confusion matrix metrics to the flattened report
    flattened_report['confusion_matrix_tn'] = tn
    flattened_report['confusion_matrix_fp'] = fp
    flattened_report['confusion_matrix_fn'] = fn
    flattened_report['confusion_matrix_tp'] = tp
    
    # include the model name as a feature
    flattened_report = {'model_name': model_name, **flattened_report}

    # write the flattened data to a csv file
    file_exists = os.path.isfile(output_file)
    
    with open(output_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=flattened_report.keys())
        # write the header only if the file does not exist
        if not file_exists:
            writer.writeheader()
        # write the data
        writer.writerow(flattened_report)

    print(f"Metrics have been saved to {output_file}")
    
evaluate_and_save_classification_results("Model", y_test, y_pred)
